In [ ]:
import os
import gdal
import numpy as np
import os, shutil
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2
from skimage import io 
import datetime
from dateutil.parser import parse
import re
from time import gmtime, strftime
import time
import schedule

gdal.UseExceptions()

inputFilePath = "/mnt/FY4A/temp/"
outputPathToFogDetection = "/mnt/FY4A/final product/Fog detection/"
outputPathToDayMicrophysics = "/mnt/FY4A/final product/Day Microphysics/"

#Преобразование функцией Харрингтона для среднего и СКО
def corrBritness(massiv):
    massiv = ((massiv-massiv.min())*255) / (massiv.max()-massiv.min())
    stdr = np.std(massiv)
    meanr = np.mean(massiv)
    v=massiv.shape
    width = v[0]
    height = v[1]
    massivNew = np.zeros((width, height), 'uint8')
    for i in range(0, width):
        for j in range(0, height):
            massivNew[i,j]=np.exp(-np.exp(-(massiv[i,j]-meanr+stdr/2)/stdr*1.5))*220
    return massivNew

def get_masked_file(pathToFile):
    pattern = ["^(?P<ftype>[A-Z]{2}[0-9]{1}[A-Z]{1})", 
           "[-]{1}[_]{1}",                
           "([A-Z]{4})",                  
           "[-]{2}[_]{1}",                
           "([A-Z]{1})",                  
           "[_]{1}",                      
           "([A-Z]{4})",                  
           "[_]{1}", 
           "([0-9]{4}[A-Z]{1})", 
           "[_]{1}", 
           "([A-Z]{1}[0-9]{1})", 
           "[-]{1}[_]{1}", 
           "([A-Z]{3})", 
           "[-]{1}[_]{1}", 
           "([A-Z]{4})", 
           "[_]{1}", 
           "([A-Z]{3})", 
           "[_]{1}", 
           "(?P<start_data>[0-9]{14})", 
           "[_]{1}", 
           "(?P<stop_data>[0-9]{14})", 
           "[_]{1}", 
           "([0-9]{4}[A-Z]{1})", 
           "[_]{1}", 
           "([A-Z]{1}[0-9]{4})", 
           "[.]{1}", 
           "([A-Z]{3})$"]

    pattern = ''.join(pattern)
    return re.search(pattern, pathToFile)

def openLastFile(file):
    ds = gdal.Open(file, gdal.GA_ReadOnly)
    first = ds.GetRasterBand(1).ReadAsArray() 
    second = ds.GetRasterBand(2).ReadAsArray()
    third = ds.GetRasterBand(3).ReadAsArray()
    fourth = ds.GetRasterBand(4).ReadAsArray() 
    fifth = ds.GetRasterBand(5).ReadAsArray()
    sixth = ds.GetRasterBand(6).ReadAsArray() 
    seventh = ds.GetRasterBand(7).ReadAsArray()
    eighth = ds.GetRasterBand(8).ReadAsArray() 
    ninth = ds.GetRasterBand(9).ReadAsArray()
    tenth = ds.GetRasterBand(10).ReadAsArray() 
    eleventh = ds.GetRasterBand(11).ReadAsArray()
    twelfth = ds.GetRasterBand(12).ReadAsArray()
    thirteenth = ds.GetRasterBand(13).ReadAsArray()
    fourteenth = ds.GetRasterBand(14).ReadAsArray()
    del ds
    return first,second,third,fourth,fifth,sixth,seventh,eighth,ninth,tenth,eleventh,twelfth,thirteenth,fourteenth

def getLastFile():
    folders = os.listdir(inputFilePath)
    files = [os.path.join(inputFilePath, file) for file in folders]
    lastFolder = max(files, key=os.path.getctime)
    print(lastFolder)
    fileName = os.listdir(lastFolder)[0]
    file = lastFolder+"/"+fileName
    return file, fileName
    
def make_fog_detection(fifth, second,fileName,pathToFile):
    """
    Проверка директории для каждого продукта
    """
    directory = outputPathToFogDetection+pathToFile.split("/")[4]+"/"
    if not os.path.exists(directory):
        try:
            os.mkdir(directory)
            print ("Успешно создана директория %s " % directory)
        except OSError as e:
            print ("Создать директорию %s не удалось" % directory, str(e))
            return
    else:
        print ("Директория уже есть %s " % directory)
        return
        
    filename = os.path.basename(pathToFile)
    index = filename.index('.')
    
    date = datetime.datetime.strptime(filename.split("_")[10], '%Y%m%d%f')
    date = date.strftime("%d.%m.%Y %I:%M %p %Z") # Выходная дата для печати в файл
    
    ouputFile = directory+filename[:index]+".jpg"
    print(ouputFile)
    
    """
    Цветосинтезированное изображение для
    обнаружения тумана с использованием
    видимого канала высокого разрешения (ВКВР)
    """
    r = corrBritness(fifth)
    g = corrBritness(second)
    b = corrBritness(second)
    
    width,height=g.shape
    rgb_array = np.zeros((width, height, 3), 'uint8')
    for x in range(0, width):
        for y in range(0, height):
            rgb_array[x, y, 0] = r[x, y] #R
            rgb_array[x, y, 1] = g[x, y] #G
            rgb_array[x, y, 2] = b[x, y] #B
    img = Image.fromarray(rgb_array).convert('RGB')
    del r,g,b,width,height,rgb_array
    
    makeFinalImage(img, ouputFile, date)


def make_analysis_of_cloud_microphysics(third,seventh,twelfth,pathToFile):
    """
    Проверка директории для каждого продукта
    """
    directory = outputPathToDayMicrophysics+pathToFile.split("/")[4]+"/"
    if not os.path.exists(directory):
        try:
            os.mkdir(directory)
            print ("Успешно создана директория %s " % directory)
        except OSError as e:
            print ("Создать директорию %s не удалось" % directory, str(e))
            return
    else:
        print ("Директория уже есть %s " % directory)
        return
        
    filename = os.path.basename(pathToFile)
    index = filename.index('.')

    date = datetime.datetime.strptime(filename.split("_")[10], '%Y%m%d%f')
    date = date.strftime("%d.%m.%Y %I:%M %p %Z") # Выходная дата для печати в файл
    
    ouputFile = directory+filename[:index]+".jpg"
    print(ouputFile)
    
    """
    Цветосинтезированное изображение для
    анализа микрофизики облаков
    в дневное время
    """
    r = corrBritness(third)
    g = corrBritness(seventh)
    b = corrBritness(twelfth)
    
    width,height=g.shape
    rgb_array = np.zeros((width, height, 3), 'uint8')
    for x in range(0, width):
        for y in range(0, height):
            rgb_array[x, y, 0] = r[x, y] #R
            rgb_array[x, y, 1] = g[x, y] #G
            rgb_array[x, y, 2] = b[x, y] #B
    img = Image.fromarray(rgb_array).convert('RGB')
    del r,g,b,width,height,rgb_array
    
    makeFinalImage(img, ouputFile, date)
    print("Цветосинтезированное изображение дляанализа микрофизики облаков в дневное время: сделано")
    
def makeFinalImage(img, ouputFile, writeData):
    substrate = cv2.imread("frame.png", cv2.IMREAD_UNCHANGED)
    hh, ww, cc = substrate.shape
    
    open_cv_image = np.array(img) 
    trees = open_cv_image
    alpha = substrate[:,:,3] 
    img = substrate[:,:,0-2]
    img[alpha==0] = 255
    img = cv2.merge((img,img,img))
    ret, thresh = cv2.threshold(alpha,0,255,0)
    thresh = 255 - thresh
    kernel = np.ones((9,9), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    cntrs = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntrs = cntrs[0] if len(cntrs) == 2 else cntrs[1]
    for c in cntrs:
        x,y,w,h = cv2.boundingRect(c)
    trees = cv2.resize(trees,(w,h),0,0)
    img = substrate[:,:,[0,1,2]]
    result = img.copy()
    result[y:y+h, x:x+w] = img[y:y+h, x:x+w]/255 * trees
    
    cv2.putText(result, str(writeData), (482,1147), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    cv2.imwrite(ouputFile, result)
    
    
def main():
    pathToFile, fileName = getLastFile()
    first,second,third,fourth,fifth,sixth,seventh,eighth,ninth,tenth,eleventh,twelfth,thirteenth,fourteenth = openLastFile(pathToFile)
    # 5 2 2
    make_fog_detection(fifth, second, fileName, pathToFile)
    # 3 8 12
    make_analysis_of_cloud_microphysics(third,eighth,twelfth, pathToFile)
    print("Завершение работы программы", datetime.datetime.now())

#Test start
main()
if __name__ == "__main__":
    print("Start app", datetime.datetime.now())
    schedule.every(4).minutes.do(main)
    while(True):
        schedule.run_pending()
        time.sleep(5)

/mnt/FY4A/temp/20191217
Директория уже есть /mnt/FY4A/final product/Fog detection/20191217/ 
Директория уже есть /mnt/FY4A/final product/Day Microphysics/20191217/ 
Завершение работы программы 2019-12-26 08:59:11.014392
Start app 2019-12-26 08:59:11.015717
/mnt/FY4A/temp/20191217
Директория уже есть /mnt/FY4A/final product/Fog detection/20191217/ 
Директория уже есть /mnt/FY4A/final product/Day Microphysics/20191217/ 
Завершение работы программы 2019-12-26 09:03:11.373262
/mnt/FY4A/temp/20191217
Директория уже есть /mnt/FY4A/final product/Fog detection/20191217/ 
Директория уже есть /mnt/FY4A/final product/Day Microphysics/20191217/ 
Завершение работы программы 2019-12-26 09:07:11.735992
/mnt/FY4A/temp/20191217
Директория уже есть /mnt/FY4A/final product/Fog detection/20191217/ 
Директория уже есть /mnt/FY4A/final product/Day Microphysics/20191217/ 
Завершение работы программы 2019-12-26 09:11:12.534482
/mnt/FY4A/temp/20191217
Директория уже есть /mnt/FY4A/final product/Fog detection/2